# Where Was the Image Manipulated?

In many real-world scenarios, knowing that an image is suspicious is not enough.
Investigators, researchers, and analysts often need to understand *where* the
visual inconsistency is located.

This notebook focuses on localization:
- how VAAS highlights suspicious regions,
- how anomaly maps should be interpreted,
- and how visualization parameters affect what we see.

This notebook assumes familiarity with basic VAAS inference.


## Installation

If VAAS is not already installed in your environment, install it via pip.


In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO

## Loading the Pretrained VAAS Model

We load the same pretrained checkpoint used in the quick-start notebook.
The model is placed on CPU for maximum compatibility.

In [ ]:
pipeline = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cpu",
    alpha=0.5,
)

## Loading an Example Image

The image below is loaded directly from the VAAS GitHub repository.
Using online images ensures this notebook runs without additional setup.

In [ ]:
image_url = (
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/"
    "examples/images/COCO_DF_C110B00000_00539519.jpg"
)

response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

image

## Running Inference

Before visualizing suspicious regions, we run standard VAAS inference.
This produces the anomaly map used for localization.

In [ ]:
result = pipeline(image)

## What Is the Anomaly Map?

The anomaly map is a spatial representation of visual inconsistency.
Each pixel value reflects how strongly that region contributes to the
overall anomaly score.

Important points:
- The anomaly map is resized to 224 × 224.
- It is a post-processed output, not raw attention weights.
- Higher values indicate stronger deviation from learned visual patterns.

The anomaly map is most useful when interpreted qualitatively rather than
as a precise segmentation mask.

## Visualizing Suspicious Regions

VAAS provides a built-in visualization utility that overlays anomaly information
on the original image.

The visualization highlights:
- regions with high anomaly response,
- and attention patterns from the ViT backbone.

The `mode` parameter controls what is displayed.


In [ ]:
pipeline.visualize(
    image=image,
    save_path="vaas_anomaly_visualization.png",
    mode="all",
    threshold=0.5,
)

In [ ]:
from IPython.display import Image as IPImage, display
display(IPImage("vaas_anomaly_visualization.png"))

## The Role of the Threshold Parameter

The `threshold` parameter controls how aggressively suspicious regions
are highlighted.

- Lower thresholds reveal more subtle inconsistencies.
- Higher thresholds focus only on the strongest anomaly responses.

Thresholding does not change the underlying scores.
It only affects visualization.

Below, we generate multiple visualizations using different thresholds
to illustrate how localization sensitivity changes.


In [ ]:
thresholds = [0.3, 0.5, 0.7]

for t in thresholds:
    output_path = f"vaas_visualization_threshold_{t}.png"
    pipeline.visualize(
        image=image,
        save_path=output_path,
        mode="all",
        threshold=t,
    )

    display(IPImage(output_path))

## Practical Interpretation

Anomaly maps should be interpreted as guidance rather than ground truth.

In practice, they are useful for:
- directing human attention during inspection,
- prioritizing regions for further forensic analysis,
- supporting explainability in reporting and documentation.

They are not designed to replace semantic segmentation or pixel-accurate masks.

## Limitations

With the current VAAS API:
- anomaly maps are generated at a fixed resolution,
- raw attention tensors are not exposed,
- localization is descriptive rather than strictly causal.

These design choices keep the API simple and inference-focused.
Future releases may expand researcher-facing access where appropriate.


## Summary

In this notebook, we:
- examined how VAAS localizes visual inconsistencies,
- visualized anomaly maps using built-in utilities,
- and explored how thresholding affects interpretation.

The next notebook will focus on understanding how VAAS scores
(S_F, S_P, and S_H) are computed and how they relate to one another.
check it out: [03_understanding_vaas_scores_sf_sp_sh.ipynb](https://colab.research.google.com/drive/1yNKrlwue9BItzqmhZUZ4-3d5kBAm9qys?usp=sharing)
